In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (14,8)
plt.rcParams['figure.dpi'] = 150
sns.set()
sns.set_context("talk")

In [24]:
# quadrumer functions

def extract_quadrumers(aptamer_sequence):
    """takes in one 18-mer and return a table of quadrumers, with a position column and a quadrumer column"""
    
    quadrumers = []
    for i in np.arange(len(aptamer_sequence)-3): 
        quad = aptamer_sequence[i:i+4]
        quadrumers = np.append(quadrumers,quad)
    return quadrumers


def quadrumer_with_position(aptamer_sequence):
    """return a table with position and relative quadrumer sequence for a given aptamer sequence"""
    
    quadrumers = extract_quadrumers(aptamer_sequence)
    positions = np.arange(1,len(aptamer_sequence)-2)
    return pd.DataFrame({'Position':positions,'Quadrumer':quadrumers})



def full_table_quadrumer(tbl):
    """ Function that exports a Trimer and position features from raw data

        Inputs:
            tbl : a table like RnE
            
        Outputs:
            tbl: columns are 'Position' 'Trimer' and 'Count' Note: not weighted bc missing data
    """    
    total_position = []
    total_quad = []

    for i in np.arange(len(tbl)):
        temp_18mer = tbl['sequence'][i]
        temp_df = quadrumer_with_position(temp_18mer).groupby(['Position','Quadrumer']).size().to_frame().reset_index().rename(columns={0:'count'})
        total_position = np.append(total_position,temp_df['Position'])
        total_quad = np.append(total_quad,temp_df['Quadrumer'])

    return pd.DataFrame({'Position':total_position,'Quadrumer':total_quad}).astype({'Position': 'int64'})


def freq_only_quadrumer(full_weighted_tbl):
    """ Function that exports a quadrumer and position features from raw data

        Inputs:
            tbl : full table with position and quad/trimer seqence from full table function    
                       
        Outputs:
            tbl: Aggregated table with summed frequency only. index is quadrumer and col is summed count
    """       
    return full_weighted_tbl.groupby('Quadrumer').sum().reset_index().rename(columns={'Position':'Count'})



In [28]:
# functions for data processing

def extract_trimers(aptamer_sequence):
    """takes in one 18-mer and return a table of quadrumers, with a position column and a quadrumer column"""
    
    trimers = []
    for i in np.arange(len(aptamer_sequence)-2):
        tri = aptamer_sequence[i:i+3]
        trimers = np.append(trimers,tri)
    return trimers

def trimer_with_position(aptamer_sequence):
    """return a table with position and relative quadrumer sequence for a given aptamer sequence"""
    
    trimers = extract_trimers(aptamer_sequence)
    positions = np.arange(1,len(aptamer_sequence)-1)
    return pd.DataFrame({'Position':positions,'Trimer':trimers})

def full_table_trimer(tbl):
    """ Function that exports a Trimer and position features from raw data

        Inputs:
            tbl : a table like RnE
            
        Outputs:
            tbl: columns are 'Position' 'Trimer' and 'Count' Note: not weighted bc missing data
    """    
    total_position = []
    total_tri = []

    for i in np.arange(len(tbl)):
        temp_18mer = tbl['sequence'][i]
        temp_df = trimer_with_position(temp_18mer).groupby(['Position','Trimer']).size().to_frame().reset_index().rename(columns={0:'count'})
        total_position = np.append(total_position,temp_df['Position'])
        total_tri = np.append(total_tri,temp_df['Trimer'])

    return pd.DataFrame({'Position':total_position,'Trimer':total_tri}).astype({'Position': 'int64'})


def freq_only_trimer(full_weighted_tbl):
    """ Function that exports a Trimer and position features from raw data

        Inputs:
            tbl : full table with position and quad/trimer seqence from full table function    
                       
        Outputs:
            tbl: Aggregated table with summed frequency only. index is trimer and col is summed count
    """       
    return full_weighted_tbl.groupby('Trimer').sum().reset_index().rename(columns={'Position':'Count'})


# not usable bc no RPM
def wise_append(original_list, temp_list):
    if len(temp_list) == 0:
        return np.append(original_list, 0)
    else:
        return np.append(original_list, temp_list[0])
    

def expand_df(df):
    """takes a df with quads as index, and one 'R2E count' column and one 'R6E count' columns
    #return a expand it to R3,R4, and R5 using the given index as reference."""
    R3E_count_6rich2low = []
    R4E_count_6rich2low = []
    R5E_count_6rich2low = []

    for i in df.index:
        R3E_temp_6rich2low = R3E_frequency.query("Quadrumer == @i")['Weighted frequency']
        R4E_temp_6rich2low = R4E_frequency.query("Quadrumer == @i")['Weighted frequency']
        R5E_temp_6rich2low = R5E_frequency.query("Quadrumer == @i")['Weighted frequency']
    
        R3E_count_6rich2low = wise_append(R3E_count_6rich2low,R3E_temp_6rich2low)
        R4E_count_6rich2low = wise_append(R4E_count_6rich2low,R4E_temp_6rich2low)
        R5E_count_6rich2low = wise_append(R5E_count_6rich2low,R5E_temp_6rich2low)
    
    return pd.DataFrame({'R2E':df['R2E count'],
                                        'R3E':R3E_count_6rich2low,
                                        'R4E':R4E_count_6rich2low,
                                        'R5E':R5E_count_6rich2low,
                                        'R6E':df['R6E count'],
                                       })

def expand_df_ctrl(df):
    """ takes a df with quads as index, and one 'R2E count' column and one 'R6E count' columns
    return a expand it to R3,R4, and R5 using the given index as reference. """
    
    R3C_count_6rich2low = []
    R4C_count_6rich2low = []
    R5C_count_6rich2low = []

    for i in df.index:
        R3C_temp_6rich2low = R3C_frequency.query("Quadrumer == @i")['Weighted frequency']
        R4C_temp_6rich2low = R4C_frequency.query("Quadrumer == @i")['Weighted frequency']
        R5C_temp_6rich2low = R5C_frequency.query("Quadrumer == @i")['Weighted frequency']
    
        R3C_count_6rich2low = wise_append(R3C_count_6rich2low,R3C_temp_6rich2low)
        R4C_count_6rich2low = wise_append(R4C_count_6rich2low,R4C_temp_6rich2low)
        R5C_count_6rich2low = wise_append(R5C_count_6rich2low,R5C_temp_6rich2low)
    
    return pd.DataFrame({'R2C':df['R2C count'],
                                        'R3C':R3C_count_6rich2low,
                                        'R4C':R4C_count_6rich2low,
                                        'R5C':R5C_count_6rich2low,
                                        'R6C':df['R6C count'],
                                       })

**Import raw data**

In [2]:
repo_url = 'https://raw.githubusercontent.com/Xiaoqi-Sun/aptamer_scoring/main/'

In [4]:
# # Serotonin (Sequence, Rank, Reads, RPM, Trimed)

# for i in np.arange(2,7):
#     exec("R{}E = pd.read_csv(repo_url+'serotonin%20raw%20data/{}RE.csv')".format(i, i))
#     exec("R{}C = pd.read_csv(repo_url+'serotonin%20raw%20data/{}RC.csv')".format(i, i))

In [5]:
# # Oxytocin (Sequence, Reads, RPM)

# for i in np.arange(3,7):
#     exec("R{}E_O = pd.read_csv(repo_url+'Oxytocin%20raw%20data/R{}E_O.csv',header=0,names=['Trimed', 'Reads', 'RPM'])".format(i, i))

In [3]:
# NPY

R4 = pd.read_excel('/Users/xiaoqisun/Desktop/Seven/UCB/Research/Jaquesta/npyaptamers.xlsx',sheet_name=0,header=None,names=['name','sequence'])
R6 = pd.read_excel('/Users/xiaoqisun/Desktop/Seven/UCB/Research/Jaquesta/npyaptamers.xlsx',sheet_name=1,header=None,names=['name','sequence'])

**Data processing:** processed data are saved into csv files

In [6]:
# Serotonin

# for i in np.arange(2,7):
#    exec('R{}E_full_table_weighted = full_table_weighted(R{}E)'.format(i, i))
#    exec('R{}E_frequency = weighted_freq_only(R{}E_full_table_weighted)'.format(i, i))
#    exec('R{}C_full_table_weighted = full_table_weighted(R{}C)'.format(i, i))
#    exec('R{}C_frequency = weighted_freq_only(R{}C_full_table_weighted)'.format(i, i))

In [17]:
# Oxytocin

# for i in np.arange(3,7):
#    exec('R{}E_O_full_table_weighted = full_table_weighted(R{}E_O)'.format(i, i))
#    exec('R{}E_O_frequency = weighted_freq_only(R{}E_O_full_table_weighted)'.format(i, i))

In [32]:
# NPY - trimer
R4_full_table_trimer = full_table_trimer(R4)
R4_frequency_trimer = freq_only_trimer(R4_full_table_trimer)
R6_full_table_trimer = full_table_trimer(R6)
R6_frequency_trimer = freq_only_trimer(R6_full_table_trimer)


# NPY - quadrumer
R4_full_table_quadrumer = full_table_quadrumer(R4)
R4_frequency_quadrumer = freq_only_quadrumer(R4_full_table_quadrumer)
R6_full_table_quadrumer = full_table_quadrumer(R6)
R6_frequency_quadrumer = freq_only_quadrumer(R6_full_table_quadrumer)

**Export processed data:** all RnE_frequency and full_table_weighted

In [7]:
# Serotonin

#for i in np.arange(2,7):
#    exec("R{}E_frequency.to_csv('R{}E_frequency.csv', index=True)".format(i, i))
#    exec("R{}E_full_table_weighted.to_csv('R{}E_full_table_weighted.csv', index=True)".format(i, i))
#    exec("R{}C_frequency.to_csv('R{}C_frequency.csv', index=True)".format(i, i))
#    exec("R{}C_full_table_weighted.to_csv('R{}C_full_table_weighted.csv', index=True)".format(i, i))

In [20]:
# Oxytocin

# for i in np.arange(3,7):
#    exec("R{}E_O_frequency.to_csv('R{}E_O_frequency.csv', index=True)".format(i, i))
#    exec("R{}E_O_full_table_weighted.to_csv('R{}E_O_full_table_weighted.csv', index=True)".format(i, i))

In [35]:
# NPY

R4_full_table_quadrumer.to_csv('R4_full_table.csv', index=True)  
R4_frequency_quadrumer.to_csv('R4_frequency.csv', index=True)  
R6_full_table_quadrumer.to_csv('R6_full_table.csv', index=True) 
R6_frequency_quadrumer.to_csv('R6_frequency.csv', index=True) 

**Re-import processed data:**  the location is moved

In [30]:
# Serotonin

for i in np.arange(2,7):
    exec("R{}E_frequency = pd.read_csv(repo_url+'serotonin%20data%20processed/R{}E_frequency.csv',index_col='Quadrumer')".format(i, i))
    exec("R{}C_frequency = pd.read_csv(repo_url+'serotonin%20data%20processed/R{}C_frequency.csv',index_col='Quadrumer')".format(i, i))
    exec("R{}E_full_table_weighted = pd.read_csv(repo_url+'serotonin%20data%20processed/R{}E_full_table_weighted.csv',index_col=0)".format(i, i))
    exec("R{}C_full_table_weighted = pd.read_csv(repo_url+'serotonin%20data%20processed/R{}C_full_table_weighted.csv',index_col=0)".format(i, i))

In [4]:
# Oxytocin

for i in np.arange(3,7):
    exec("R{}E_O_frequency = pd.read_csv(repo_url+'Oxytocin%20processed%20data/R{}E_O_frequency.csv',index_col='Quadrumer')".format(i, i))
    exec("R{}E_O_full_table_weighted = pd.read_csv(repo_url+'Oxytocin%20processed%20data/R{}E_O_full_table_weighted.csv',index_col=0)".format(i, i))

In [31]:
print('There are', len(R6E_frequency.index.unique()),'unique quadrumers in R6E')
print('There are', len(R6C_frequency.index.unique()),'unique quadrumers in R6C')

There are 228 unique quadrumers in R6E
There are 253 unique quadrumers in R6C


In [9]:
R6E_top10 = R6E_frequency.sort_values('Weighted frequency',ascending=False).head(10)
R6E_top10

,Quadrumer,Weighted frequency
17,ACAC,955.754268
66,CACA,675.415167
67,CACC,518.755081
20,ACCA,411.682731
79,CCAC,359.133666
197,TCCG,319.881547
63,CAAC,276.390663
69,CACT,271.687377
86,CCGA,260.364650
141,GCAC,258.738822


In [30]:
R6C_top10 = R6C_frequency.sort_values('Weighted frequency',ascending=False).head(10)
R6C_top10

,Quadrumer,Weighted frequency
109,CGTG,276.067596
17,ACAC,269.934825
133,GACG,240.693876
237,TGTG,225.031924
27,ACGT,213.980331
96,CGAC,205.721488
184,GTGC,203.953631
69,CACG,193.253619
67,CACA,190.543790
22,ACCG,190.076121
